JUPYTER NOTEBOOK FOR FELINE DATA ANALYSIS

In [1]:

import numpy as np
import pandas as pd
%matplotlib inline

import os
os.getcwd()

Cranial = pd.read_csv('/Users/88firefly/Feline-Bones-Final-Project/data/Felidae Cranial Data.csv',delimiter=',')
Forelimb= pd.read_csv('/Users/88firefly/Feline-Bones-Final-Project/data/Felidae and Canidae postcranial database.csv',delimiter=',')

Forelimb_Data=Forelimb.values[:261,:] #cut out dogs

Forelimb_genusID=Forelimb_Data[:,2]
Forelimb_speciesID=Forelimb_Data[:,3]
Forelimb_sexID=Forelimb_Data[:,4]

Cranial_Data=Cranial.values

Cranial_genusID=Cranial_Data[:,2]
Cranial_speciesID=Cranial_Data[:,3]
Cranial_sexID=Cranial_Data[:,4]


print("The Cranial dataset has", np.unique(Cranial[['Species']]).shape, "unique species, and", Cranial.shape, "samples.")
print("The Forelimb dataset has", np.unique(Forelimb[['Species']]).shape, "unique species, and", Forelimb.shape, "samples.")


type(Cranial_speciesID)

The Cranial dataset has (77,) unique species, and (338, 52) samples.
The Forelimb dataset has (95,) unique species, and (323, 61) samples.


numpy.ndarray

In [2]:
Forelimb.shape

(323, 61)

In [3]:
ComboData_All=Cranial.merge(Forelimb,how='outer', on=None, left_on='Catalog no.', right_on='Catalog no.')
ComboData_Shared=Cranial.merge(Forelimb,how='inner', on=None, left_on='Catalog no.', right_on='Catalog no.')
print(ComboData_Shared.head)

<bound method NDFrame.head of     Museum_x Catalog no.       Genus_x                Species_x    Sex_x  \
0        MVZ      184066      Acinonyx                  jubatus  unknown   
1       FMNH       34589      Acinonyx                  jubatus     male   
2       FMNH       34589      Acinonyx                  jubatus     male   
3       FMNH      135042      Caracal                   caracal  unknown   
4       USNM      384162      Caracal                   caracal   female   
5       FMNH       72804      Catopuma               temminckii   female   
6       USNM      339645      Catopuma               temminckii  unknown   
7       USNM      362188      Catopuma               temminckii   female   
8       USNM      395843      Catopuma               temminckii   female   
9       FMNH       98908      Catopuma               temminckii     male   
10      USNM      176049         Felis            chaus affinis   female   
11      FMNH      159995         Felis                marg

In [4]:
ComboShared_Data=ComboData_Shared.values
ComboAll_Data=ComboData_All.values

In [5]:
from sklearn.preprocessing import Imputer


ComboShared_NumData=np.delete(ComboShared_Data,np.arange(52,58),axis=1)#52-57 unnecessary repeats


ComboShared_speciesID=ComboShared_NumData[:,3]
ComboShared_genusID=ComboShared_NumData[:,2]
ComboShared_sexID=ComboShared_NumData[:,4]

In [6]:
imp=Imputer(missing_values='NaN', strategy='mean', axis=0, copy=True)
Cranial_filled_data=imp.fit_transform(Cranial_Data[:,7:])
Forelimb_filled_data=imp.fit_transform(Forelimb_Data[:,7:])
ComboShared_filled_data=imp.fit_transform(ComboShared_NumData[:,7:])


In [7]:
from sklearn.decomposition import PCA

def test_train_Graphpca(original_pf, df_value, df_ind, test_percent):
    #create test and train indices
    L = len(df_value)
    train_ind = np.random.choice(np.arange(0,len(df_ind)), size=(len(df_ind)*test_percent), replace=False, p=None)
    test_ind = np.setdiff1d(np.arange(0,L), train_ind)
    
    #run pca on test data
    test_pca = PCA()
    test_pca.fit(df_value[train_ind,:], df_ind[train_ind])
    explained_variance = test_pca.explained_variance_ratio_
    
    #first and second component graph
    value_pca = test_pca.transform(df_value)
    for sp in set(df_ind):
        myspecies = original_pf.str.contains(sp)
        p = plt.plot(value_pca[np.ix_(myspecies), 0].flatten(),
            value_pca[np.ix_(myspecies), 1].flatten(),
            '.', label = sp)
        plt.xlabel('PC1')
        plt.ylabel('PC2')
        plt.legend(loc = 'center left',bbox_to_anchor=(1, .5))
    return(p)


In [25]:
def test_train(df_value, df_ind, test_percent):
    #create test and train indices
    L = len(df_value)
    train_ind = np.random.choice(np.arange(0,len(df_ind)), size=(len(df_ind)*test_percent), replace=False, p=None)
    test_ind = np.setdiff1d(np.arange(0,L), train_ind)
    
    test = df_value[test_ind,:]
   
    train = df_value[train_ind,:]
    return train,test

In [35]:
from sklearn.neighbors import KNeighborsClassifier
svc_model=identify = SVC()

kn_model=KNeighborsClassifier()


def cross_val_class_accuracy(model, df_value, df_ind, r, test_percent, reps):
    test_pca = PCA()
    
    score = np.array([])
    for i in range(reps):
        train, test = test_train(df_value, df_ind, test_percent)
        y_train = df_ind[train_ind]
        y_test = df_ind[test_ind]
        
        X_train = test_pca.fit_transform(train)[:, :r]
        X_test = test_pca.transform(test)[:, :r]
        
        model.fit(X_train, y_train)
        
        pred = model.predict(X_test)
        this_score = sum(pred == y_test) / len(y_test)
        
        score = np.append(score, this_score)
        avg_score=np.mean(score)
    return avg_score

cross_val_class_accuracy(kn_model, Cranial_filled_data, Cranial_speciesID, 10, 0.8, 100)

/Users/88firefly/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


0.021323529411764706

In [26]:







from sklearn.svm import SVC

test_train(Cranial_filled_data, Cranial_speciesID, 0.8)

L = len(Cranial_speciesID)
train_ind = np.random.choice(np.arange(0,len(Cranial_speciesID)), size=(len(Cranial_speciesID)*0.8), replace=False, p=None)
test_ind = np.setdiff1d(np.arange(0,L), train_ind)
    
test = Cranial_filled_data[test_ind,:]
   
train = Cranial_filled_data[train_ind,:]


train_Ind=train[:,np.newaxis]
test_Ind=test[:,np.newaxis]

test_pca = PCA()

X_train = test_pca.fit_transform(train)[:, :10]
X_test = test_pca.transform(test)[:, :10]

y_train = Cranial_speciesID[train_ind]
y_test = Cranial_speciesID[test_ind]

identify = SVC()


# identify.fit(*training data*, *training ids*) hint: *training data* <= Swirl[train_ind,:]
identify.fit(X_train,y_train)
# predicted = identify.predict(*test data*)
predicted=(identify.predict(X_test))

predicted==Cranial_speciesID[test_ind]
np.mean(predicted==Cranial_speciesID[test_ind])

/Users/88firefly/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/88firefly/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


0.029411764705882353